In [1]:
!pip install -r requirements.txt

In [2]:
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle #We can reuse this when we are doing the deployment

In [4]:
data=pd.read_csv("Churn_Modelling.csv")

In [5]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
#The rownumber, customerid and surname are not important features

In [7]:
#Preprocessing the data by droppingthe irrelevant features

In [8]:
data=data.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [9]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [10]:
#Geography and Gender are categorical variables

In [11]:
#I am going to apply encoding for categorical variables

In [12]:
label_encoder_gender=LabelEncoder()

In [13]:
data["Gender"]=label_encoder_gender.fit_transform(data["Gender"])

In [14]:
data["Gender"].unique()

array([0, 1])

In [15]:
data["Geography"].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [16]:
data["Geography"].nunique()

3

In [17]:
data["Geography"].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [18]:
#I am using OHE instead of label encoding on Geography column as I don't want spain to have more valur than germany or vice versa

In [19]:
#OHE Geography
from sklearn.preprocessing import OneHotEncoder

In [20]:
one_hot_encoder_geo= OneHotEncoder(sparse_output=False) #sparse_output=False converts to 2d numpy array which contains zero elements, 
#not memory efficient but suitable for smaller datasets
#When you set sparse_output=True in OneHotEncoder, the output is a sparse matrix instead of a dense NumPy array. 
#This means the encoder stores only nonzero values and their positions, instead of storing the entire matrix with zeros.

In [21]:
geo_encoder=one_hot_encoder_geo.fit_transform(data[["Geography"]]) #The reason for using [[]] is because OHE returns a 2d dataframe, not a 1d list.

In [22]:
geo_encoder.shape

(10000, 3)

In [23]:
one_hot_encoder_geo.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [24]:
geo_encoded_df=pd.DataFrame(geo_encoder,columns=one_hot_encoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [25]:
#Combine OHE columns with the original data

In [26]:
data=pd.concat([data.drop("Geography", axis=1), geo_encoded_df] ,axis=1)

In [27]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [28]:
#Saving the encoders with pickle file
with open("label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender,file)

with open("one_hot_encoder_geo.pkl","wb") as file:
    pickle.dump(one_hot_encoder_geo,file)

In [29]:
#Dividing the dataset into independent and dependent features

In [30]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [31]:
#Exit will be the dependent feature and the rest will be independent

In [32]:
X=data.drop(["Exited"],axis=1)

In [33]:
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,0.0,1.0,0.0


In [34]:
y=data["Exited"]

In [35]:
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [36]:
#Now splitting the data into training and testing sets:

In [37]:
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.2,random_state=42)
#Scale the features:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [38]:
(X_train.shape[1],)

(12,)

In [39]:
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

In [40]:
#ANN Implementation


In [41]:
import tensorflow as tf

In [42]:
#When we start creating our ANN, we have to initialize sequetial network
#In keras we use class Dense for creating hidden layers

In [43]:
from tensorflow.keras.models import Sequential

In [44]:
from tensorflow.keras.layers import Dense

In [45]:
from tensorflow.keras.layers import Input

In [46]:
from tensorflow.keras.callbacks import EarlyStopping

In [47]:
from tensorflow.keras.callbacks import TensorBoard

In [48]:
import datetime

In [49]:
X_train.shape

(8000, 12)

In [50]:
#Build our ANN model:
model=Sequential([
    #I need to check the shape of my inputs
    Input(shape=(X_train.shape[1],)),
    Dense(64,activation="relu"), #This is the first hidden layer connected to the input layer
    #For second hidden layer we don't have to write the input shape, it is needed only for the first
    Dense(32,activation="relu"), #HL2
    Dense(1,activation="sigmoid")
])

In [51]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [52]:
#Another way to compile the model 
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.1)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x000001C1D7FC2B00>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [53]:
#compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=["accuracy"])

In [54]:
#Setup the tensorboard

In [69]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir="logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [70]:
#Setup EarlyStopping:
early_stopping_callback=EarlyStopping(monitor="val_loss",patience=25,restore_best_weights=True)

In [71]:
history=model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=100,callbacks=[tensorflow_callback,early_stopping_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8539 - loss: 0.3824 - val_accuracy: 0.7985 - val_loss: 0.4445
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8495 - loss: 0.3804 - val_accuracy: 0.8590 - val_loss: 0.4561
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8562 - loss: 0.3737 - val_accuracy: 0.8595 - val_loss: 0.3679
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8557 - loss: 0.3750 - val_accuracy: 0.8585 - val_loss: 0.3817
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8579 - loss: 0.3617 - val_accuracy: 0.8595 - val_loss: 0.3664
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8539 - loss: 0.3773 - val_accuracy: 0.8595 - val_loss: 0.3669
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8559 - loss: 0.3620 - val_accuracy: 0.8610 - val_loss: 0.3838
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8501 - loss: 0.3864 - val_accu

In [72]:
#This only ran to 7 epochs for patience=5, now let's increase it to 10 and see the results:

In [73]:
model.save("model.h5")

In [77]:
#Load Tensorboard extenssion
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [83]:
%tensorboard --logdir logs/fit --port=6007

In [80]:
%reload_ext tensorboard
